# Image Rectification
The goal of image rectificiation is to find the Homographies H and H' for the stereo images such that in the transformed image pai, the same world point appears on the same row.

### Import Statements

In [1]:
import cv2
import numpy as np
from scipy.optimize import least_squares
from image_rectification_helper import *

### Load Images

In [2]:
left_file_path = "/home/jo_wang/Desktop/ECE661/HW09/input_images/left.jpg"
right_file_path = "/home/jo_wang/Desktop/ECE661/HW09/input_images/right.jpg"
left_img = cv2.imread(left_file_path, 1)
right_img = cv2.imread(right_file_path, 1)
assert(left_img.shape == right_img.shape)

### Point Correspondences
Purpose: Establish Point 8 point correspondences between left and right image
* x_points (list): list of points in left image
* x_points_prime (list): list of points in right image
* Both lists look like this:
[[x1,y1], [x2,y2], [x3,y3], ..., [x8,y8]]

In [3]:
point_corr_image  = np.hstack((left_img, right_img))
width_adj = right_img.shape[1]
x_points = [[722, 347], [690, 466], [820, 338], [785, 453], [284,487], [280, 463], [477, 191], [622,185]]
x_prime_points = [[618,428], [599, 531], [750, 433], [717, 537], [265,457], [274,430], [505,182], [674,184]]
rainbow = [(211, 0, 148), (130, 0, 75), (255, 0, 0), (0, 255, 0), (0, 255, 255), (0, 127, 255), (0, 0, 255), (0, 0, 0)]

for i in range(len(x_points)):
    left_point = tuple(x_points[i])
    right_point = (x_prime_points[i][0] + width_adj, x_prime_points[i][1])
    cv2.circle(point_corr_image, left_point, 4, rainbow[i], -1)
    cv2.circle(point_corr_image, right_point, 4, rainbow[i], -1)
    cv2.line(point_corr_image, left_point, right_point, rainbow[i], 1)

# cv2.imwrite("point_correspondences.jpg", point_corr_image)

### Normalized 8-point Algorithm for F
Purpose: Given 8 image point correspondences (x, x'), determine the fundamental matrix F

Algorithm:
1. Normalization: Transform the image coordinates to: $\hat x_i = Tx_i$ and $\hat x'_i = T'x'_i$

    T = $\begin{bmatrix}
            s & 0 & -s\bar x\\
            0 & s & -s\bar y\\
            0 & 0 & 1\\
        \end{bmatrix}$
    $\bar x$ and $ \bar y$ : mean of x and y coordinates respectively
    $D$ : list of distances from each point to $(\bar x,\bar y)$
    $\bar D$: mean of D
    $s = \frac{\sqrt 2}{\bar D}\\$

2. Find $\hat F$ from $A \hat F = 0$

   $A \in \mathbb{R}^{8 \times 9}$, $\hat F \in \mathbb{R}^{9}$

3. Constraint Enforcement: Ensure that the rank($\hat F$) = 2

   $\hat F = UDV^T$, $D = diag(r,s,t)$
   $ \hat F' = U diag(r,s,0) V^T$

4. Denormalization: $F = T'^T \hat F' T$

In [4]:
F = compute_F(x_points, x_prime_points)
assert(np.linalg.matrix_rank(F) == 2)
print(f'Fundamental Matrix F: \n{F}')

Fundamental Matrix F: 
[[ 1.35574263e-10  7.36104078e-11 -6.86165608e-05]
 [ 1.82189401e-10 -3.61378824e-11 -1.30055509e-04]
 [ 4.19787748e-05  1.02180418e-04  1.00000000e+00]]


### Estimate the Left and Right Epipoles
$F\vec e = 0$
<br>
$\vec e'^T F = 0$

$\therefore \vec e $ is the right null vector and $\vec e'$ is the left null vector of $F$


In [5]:
e, e_prime = compute_e(F)
print(f'left epipole e: {e}')
print(f'right epipole e\': {e_prime}')

left epipole e: [ 6.58264939e+05 -2.80221557e+05  1.00000000e+00]
right epipole e': [-1.09955053e+06  5.87805748e+05  1.00000000e+00]


### Obtain the Initial Estimate of the Projection Matrices in Canonical From
$P =
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
\end{bmatrix}$
;
$P' =
\begin{bmatrix}
[e']_x & F & |e'\\
\end{bmatrix}$
;
$[e']_x =
\begin{bmatrix}
0 & -e'_3 & e'_2\\
e'_3 & 0 & -e'_1\\
-e'_2 & e'_1 & 0\\
\end{bmatrix}$

In [6]:
P, P_prime = compute_P(e_prime, F)
print(f'P: \n {P}\n')
print(f'P\': \n {P_prime}')

P: 
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]

P': 
 [[ 2.46753651e+01  6.00622368e+01  5.87805748e+05 -1.09955053e+06]
 [ 4.61577840e+01  1.12352532e+02  1.09955053e+06  5.87805748e+05]
 [-2.80017783e-04 -3.53319326e-06  1.83335812e+02  1.00000000e+00]]


### Perform Triangulation to find World Points from (x,x') correspondences
$A \vec X = \vec 0 \rightarrow
\begin{bmatrix}
xP_3^T - P_1^T \\
yP_3^T - P_2^T \\
x'P'_3^T - P'_1^T \\
y'P'_3^T - P'_2^T \\
\end{bmatrix}
\begin{bmatrix}
X_x\\ X_y \\ X_z \\ X_w\\
\end{bmatrix} =
\begin{bmatrix}
0 \\ 0 \\ 0 \\ 0\\
\end{bmatrix}$

$A \in \mathbb{R}^{4 \times 4}$ ; $X \in \mathbb{R}^{4}$

$X$ is given by the smallest eigenvector of $A^TA$

In [7]:
world_points = [triangulate(P, P_prime, x_points[i], x_prime_points[i]) for i in range(len(x_points))]
for i, wp in enumerate(world_points):
    print(f'World Point {i+1}: {wp}')

World Point 1: [-357284.3291963463, 326954.058014294, -20.358602671109526, 1.0]
World Point 2: [-580460.4278204894, 460159.38277500507, -25.353510760489332, 1.0]
World Point 3: [-241474.83994383493, 232629.25928583654, -15.24171215863109, 1.0]
World Point 4: [-366956.459931239, 310825.50733233686, -18.49658987776036, 1.0]
World Point 5: [-50811765.55687086, 24993675.945001416, -449.7800591048311, 1.0]
World Point 6: [-20838609.43285766, 10453084.126112135, -206.3857654292736, 1.0]
World Point 7: [-309616.9659592787, 306080.57713674544, -19.386720253667267, 1.0]
World Point 8: [-187689.94004719067, 201780.20753089542, -13.684626621475692, 1.0]


### Refinement of P'
Pack the necessary parameters into params
* packing p'

In [8]:
'''pack p' into params'''
params = np.zeros(12)
params = np.reshape(P_prime, 12)
print(f'Params: \n{params}')

Params: 
[ 2.46753651e+01  6.00622368e+01  5.87805748e+05 -1.09955053e+06
  4.61577840e+01  1.12352532e+02  1.09955053e+06  5.87805748e+05
 -2.80017783e-04 -3.53319326e-06  1.83335812e+02  1.00000000e+00]


In [9]:
'''cost_function(params, x_points, x_prime_points)
Input: params: list of parameters
       x_points: [[x1,y1], ... [x8,y8]]
       x_prime_points: [[x'1,y'1], ... [x'8,y'8]]
Output: d^2_geom
Purpose: Cost Function for LM'''
def cost_function(params, x_points, x_prime_points, world_points):
    P = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
    P_prime = np.reshape(params, (3,4))

    x_hat = [world2image(P, point) for point in world_points]
    x_prime_hat = [world2image(P_prime, point) for point in world_points]

    diff_x = np.subtract(x_points, x_hat)
    diff_x_prime = np.subtract(x_prime_points, x_prime_hat)

    cost = np.hstack((diff_x[:,0], diff_x[:,1], diff_x_prime[:,0], diff_x_prime[:,1]))
    return cost

In [10]:
optim = least_squares(cost_function, params, method='lm', args=(x_points,x_prime_points, world_points), verbose =1)
params_star = optim['x']

`ftol` termination condition is satisfied.
Function evaluations 14, initial cost 1.6001e+10, final cost 1.6001e+10, first-order optimality 6.72e+03.
